In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
ds = xr.open_dataset("with_otec/prog.nc").mean(['xh', 'yh'])
ini = xr.open_dataset("with_otec/MOM_IC.nc")

In [ ]:
# Graph the depth of each layer over time
ds['eh'] = xr.ones_like(ds['h']) * 0.5*( ds['e'].sel(zi=ds['zi'][1:]).data + ds['e'].sel(zi=ds['zi'][:-1]).data )
eh = ds['eh']
for z in ds['zl']:
    eh.sel(zl=z).plot()
    
plt.ylim(-1200, 0)
plt.title("Layer level depths vs Time")
plt.ylabel("Layer Height (m)");

### Evolution of temperature profile

In [ ]:
plt.figure(figsize=(5,5))

ini['Temp'].mean(['lath', 'lonh']).plot(y='Layer', color='k', lw=5., alpha=0.5, label="IC")
for t in ds['Time'][::5]:
    plt.plot(ds['temp'].sel(Time=t), -ds['eh'].sel(Time=t), marker=".", label=t.to_masked_array())
plt.grid(True)

plt.ylim(1200, 0)
plt.xlim(0, 30)
plt.title("Temperature Profile Near the Surface")
plt.ylabel("Depth (m)")
plt.xlabel("temp (degC)")
plt.legend();

### Show that mass sinks just shift profile towards mass source at rate of $W_{OTEC}$

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
ini['Temp'].mean(['lath', 'lonh']).plot(y='Layer', color='k', lw=5., alpha=0.5, label="IC")
for t in ds['Time'][::5]:
    W = 0.0001
    dH = W * (t - ds['Time'][0]).astype('float64')*1e-9
    plt.plot(ds['temp'].sel(Time=t), -ds['eh'].sel(Time=t) + dH, marker=".", label=t.to_masked_array())
plt.grid(True)

plt.ylim(500, 0)
plt.xlim(0, 30)
plt.title("Temperature Profile Near the Surface")
plt.ylabel("Depth (m)")
plt.xlabel("temp (degC)")
plt.legend()

plt.subplot(1,2,2)
ini['Temp'].mean(['lath', 'lonh']).plot(y='Layer', color='k', lw=5., alpha=0.5, label="IC")
for t in ds['Time'][::5]:
    W = 0.0001
    dH = W * (t - ds['Time'][0]).astype('float64')*1e-9
    plt.plot(ds['temp'].sel(Time=t), -ds['eh'].sel(Time=t) - dH, marker=".", label=t.to_masked_array())
plt.grid(True)

plt.ylim(1000, 500)
plt.xlim(0, 30)
plt.title("Temperature Profile Near the Surface")
plt.ylabel("Depth (m)")
plt.xlabel("temp (degC)")
plt.legend();

### Evolution of salinity profile

In [ ]:
plt.figure(figsize=(5,5))

ini['Salt'].mean(['lath', 'lonh']).plot(y='Layer', color='k', lw=5., alpha=0.5, label="IC")
for t in ds['Time'][::5]:
    plt.plot(ds['salt'].sel(Time=t), -ds['eh'].sel(Time=t), marker=".", label=t.to_masked_array())
plt.grid(True)

plt.ylim(1200, 0)
#plt.xlim(34.3, 35.0)
plt.title("Salinity Profile Near the Surface")
plt.ylabel("Depth (m)")
plt.xlabel("Salinity (g/kg)")
plt.legend();

# Plot Power Over Time

In [ ]:
# initialize variables as shown above
area = 1 # of the whole ocean, in m^2

rho = 1025.
cp = 3925.

# temperature at the surface
T_top = ds['temp'].sel(zl=20, method='nearest')
dT = T_top - ds['temp'].sel(zl=1000, method='nearest')

# design conditions = initial conditions at 1000m and mixed layer
# T_design = ini['Temp'][0,0]
# dT_design = T_design - ini['Temp'].sel(Layer=1000, method='nearest')[0]
# NOTE: These gave data that made no sense. Instead I now assume T_design=T and dT_design = dT
T_design = T_top
dT_design = dT

# Input w_cw in m/year.
def plotpower(w_cw, gamma, eff_tg):
    Q_cw = w_cw * area
    
    # coefficient to multiply by both powers involved
    coeff = Q_cw*rho*cp*eff_tg / (8*T_top)
    # generator
    P_gen = 3*coeff * (dT**2) / (2*(1+gamma))
    # parasitic
    P_p = coeff * dT_design**2 * (.18 + .12 * (gamma/2)**2.75)
    P_net = P_gen - P_p
    
    P_gen.plot(label='generated')
    P_p.plot(label='parasitic')
    P_net.plot(label='net power')
    plt.title('OTEC Net Generation')
    plt.ylabel('Power Intensity (W m-2)')
    plt.legend()

plotpower(.0001, 1.0, 0.85)